In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuprint import Client
import matplotlib.patheffects as path_effects
import matplotlib as mpl
import bokeh
import bokeh.palettes

from neuprint import fetch_simple_connections, fetch_synapse_connections, NeuronCriteria as NC, fetch_neurons


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + 'v1.2.1', token=auth_token)
except:
    np_client = None



In [5]:
neuron_df, roi_counts_df = fetch_neurons(NC(status='Traced'))


/Users/rhessa/miniconda3/envs/flybrainlab/lib/python3.11/site-packages/neuprint/client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


In [8]:
types = neuron_df[['type', 'instance']]
types = types.drop_duplicates().dropna()
types

,type
14,CL141
31,FS4C
39,FS4A
42,SMP505
66,FB8H
...,...
97738,CL151
97739,VP1d+VP4_l2PN2
97740,aMe6b
97766,AVLP534


In [28]:
# change viewing options to see ateast 150 rows
pd.set_option('display.max_rows', 150)

In [238]:
# no index
types[types['type'].str.contains("M_smPN")].sort_values('type').reset_index(drop=True)

,type
0,M_smPN6t2
1,M_smPNm1


In [206]:
# Drop not needed columns
roi_counts_df = roi_counts_df.drop(columns=['pre', 'post', 'downstream', 'upstream', 'mito'])
# COmbine roi df and neuron df
neuron_roi_df = neuron_df.merge(roi_counts_df, on='bodyId')
neuron_roi_df

,bodyId,instance,type,pre,post,downstream,upstream,mito,size,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,roi
0,198469830,None,None,0,0,0,0,2,6665418,Traced,True,Leaves,None,299.0,"[33045, 15655, 3280]",{},None,[],[],NotPrimary
1,198810827,None,None,0,0,0,0,1,9211633,Traced,True,Leaves,None,299.0,"[33014, 16221, 3088]",{},None,[],[],NotPrimary
2,198815212,None,None,0,0,0,0,3,27720313,Traced,True,Leaves,None,299.0,"[33342, 16128, 3216]",{},None,[],[],NotPrimary
3,199147536,None,None,0,0,0,0,3,58580023,Traced,True,Leaves,None,299.0,"[32412, 16453, 3356]",{},None,[],[],NotPrimary
4,199156210,None,None,0,0,0,0,4,82402138,Traced,True,Leaves,None,299.0,"[33401, 16741, 3240]",{},None,[],[],NotPrimary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533168,7112623023,None,None,28,17,205,17,17,26237427,Traced,True,Leaves,None,NaN,None,"{'AL(L)': {'pre': 28, 'post': 17, 'downstream'...",None,"[AL(L), AL-D(L), AL-DC1(L), AL-DC4(L), AL-DL5(L)]","[AL(L), AL-D(L), AL-DC1(L), AL-DC4(L), AL-DL5(L)]",AL-DL5(L)
533169,7112623023,None,None,28,17,205,17,17,26237427,Traced,True,Leaves,None,NaN,None,"{'AL(L)': {'pre': 28, 'post': 17, 'downstream'...",None,"[AL(L), AL-D(L), AL-DC1(L), AL-DC4(L), AL-DL5(L)]","[AL(L), AL-D(L), AL-DC1(L), AL-DC4(L), AL-DL5(L)]",NotPrimary
533170,7112623383,None,None,5,34,42,34,13,14483829,Traced,True,Leaves,None,NaN,None,"{'SNP(L)': {'pre': 5, 'post': 34, 'downstream'...",None,"[SMP(L), SNP(L)]","[SMP(L), SNP(L)]",NotPrimary
533171,7112623383,None,None,5,34,42,34,13,14483829,Traced,True,Leaves,None,NaN,None,"{'SNP(L)': {'pre': 5, 'post': 34, 'downstream'...",None,"[SMP(L), SNP(L)]","[SMP(L), SNP(L)]",SMP(L)


In [208]:
len(neuron_roi_df['type'].unique())

5621

In [227]:
# Checking for missing neuron types
checks = neuron_roi_df[['bodyId', 'type', 'roi']].drop_duplicates().dropna()
checks

,bodyId,type,roi
14,200326126,CL141,AVLP(R)
15,200326126,CL141,IB
16,200326126,CL141,ICL(R)
17,200326126,CL141,INP
18,200326126,CL141,LH(R)
...,...,...,...
533145,7112622044,LAL137,VES(R)
533146,7112622044,LAL137,VMNP
533147,7112622044,LAL137,bL(L)
533148,7112622044,LAL137,g5(R)


In [229]:
checks_al = checks[checks['roi'].str.contains("AL-")]
checks_al

,bodyId,type,roi
4020,294792184,M_vPNml53,AL-D(R)
4021,294792184,M_vPNml53,AL-DA2(R)
4022,294792184,M_vPNml53,AL-DA4m(R)
4023,294792184,M_vPNml53,AL-DC1(R)
4024,294792184,M_vPNml53,AL-DC2(R)
...,...,...,...
532174,5901222910,DM2_lPN,AL-VM5d(R)
532175,5901222910,DM2_lPN,AL-VM7d(R)
532176,5901222910,DM2_lPN,AL-VM7v(R)
532177,5901222910,DM2_lPN,AL-VP1d(R)


In [241]:
checks_al['type'].drop_duplicates().sort_values().reset_index(drop=True)  

0      AL-AST1
1        ALBN1
2        ALIN1
3        ALIN2
4        ALIN3
        ...   
438      vLN25
439      vLN26
440      vLN27
441      vLN28
442      vLN29
Name: type, Length: 443, dtype: object

In [243]:
# check for certain types
checks_al[checks_al['type'].str.contains("VP")].sort_values('type').reset_index(drop=True)

,bodyId,type,roi
0,1978908720,HRN_VP1d,AL-DM4(R)
1,1918885778,HRN_VP1d,AL-VP4(R)
2,1918885778,HRN_VP1d,AL-VP1m(R)
3,1918885778,HRN_VP1d,AL-VP1l(R)
4,1918885778,HRN_VP1d,AL-VP1d(R)
...,...,...,...
744,2069644133,VP5+_l2PN,AL-VC5(R)
745,2069644133,VP5+_l2PN,AL-VC2(R)
746,2069644133,VP5+_l2PN,AL-V(R)
747,2069644133,VP5+_l2PN,AL-VP5(R)
